# Strava GET Methods
This file has all necessary code to fetch routes for various bike races from Strava's API.

**TODO:**
- Grand Tour Stages for more recent years.
- Clean Route names, add year column to df

In [2]:
import warnings
warnings.simplefilter(action='ignore')

import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import numpy as np
import seaborn as sns
from scipy import stats
import math
import random
import matplotlib.pyplot as plt
import requests
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [3]:
#GET requests for Strava API - please don't change.
#Rate limits: 200 per 15 min and 2000 per day.
def get_request(endpoint_url):
    auth_url = "https://www.strava.com/oauth/token"
    activites_url = endpoint_url
    
    payload = {
    'client_id': "115388",
    'client_secret': '062e52684a0c4b4c7270ee392d6602d79b210d08',
    'refresh_token': '39830d6d91652b859fca9a48f724219201b1ac3e', #Fetch new one from site if 401 error code
    'grant_type': "refresh_token",
    'f': 'json'
    }
    #print("Requesting Token...\n")
    res = requests.post(auth_url, data=payload, verify=False)
    access_token = res.json()['access_token']
    #print("Access Token = {}\n".format(access_token))

    header = {'Authorization': 'Bearer ' + access_token}
    param = {'per_page': 200, 'page': 1}
    response = requests.get(activites_url, headers=header).json() #, params=param
    print(f"Done fetching route: {response['name']}")
    return response

In [4]:
cols = ['id', 'name', 'segments', 'distance', 'elevation_gain', 'description']
route_df = pd.DataFrame(columns=cols)

def add_route_to_df(response):
    var_dict = {
        'id': response['id'], 'name': response['name'],
        'segments': response['segments'], 'distance': response['distance'],
        'elevation_gain': response['elevation_gain'], 'description': response['description']
    }
    route_df.loc[len(route_df.index)] = var_dict

In [5]:
route_lst = [3017808025791303288, 3017808470760159864, 3021095816693259176, 3028722138306047882, #TDF 2023. 1-20
            3028722407588517896, 3028722782010528776, 3028723027809149962, 3052651433717029484, #TDF 2023. 1-20
            3052652189077111942, 3052652544706246276, 3052652827821399660, 3101918739797484150, #TDF 2023. 1-20
            3101919302066548602, 3101919691927650470, 3101919886819072166, 3091726474562931262, #TDF 2023. 1-20
            3091726880441816076, 3096866087557046718, 3096866815954677182, 3096866594175105820, #TDF 2023. 1-20
            14591492, 14804965, 14848530, 14848721, 14852435, 14852796, 14853298, 14853413, #Vuelta 2018. 1-21.
            14853649, 14853761, 14848115, 14898483, 14853990, 14854354, 14854653, 14854833, #Vuelta 2018. 1-21.
            14855913, 14856093, 14856187, 14898707, 14898960, #Vuelta 2018. 1-21.
            2976424897315914760, 2976425132212195336, 2976425358370076980, 2972794104523667996, #TDF 2022. 1-16.
            2972794962844504604, 2972795295082519070, 2972795730147859944, 2972796006173138460, #TDF 2022. 1-16.
            2972796253602719262, 2972796466894649886, 2972796805862740510, 2972797137020056092, #TDF 2022. 1-16.
            2972797364397059614, 2972887256422455836, 2972799227450244636, 2972887452923944478, #TDF 2022. 1-16.
            #3154929187671636870, 3154929317807231878, #TDF 2022. 16-18.
            3002208992379804442, 3002209279081703032, 3002209463024031514, 3002209760987293148, #TDF 2021. 1-20.
            3002209951081461368, 3002210161672415002, 3002221263398356600, 3002221824873239322, #TDF 2021. 1-20.
            3002224238965756536, 3002227017323538202, 3002227356721770972, 3002227574162676344, #TDF 2021. 1-20.
            3002227786208998170, 3002228060085244380, 3002228294271921628, 3002228473355258332, #TDF 2021. 1-20.
            3002228755788388828, 3002228943545318170, 3002229131710478968, 3002229388620848760, #TDF 2021. 1-20.
            #3154926582711332564, 3154927604846812014, 3154927839622753158, #Paris-Nice 2023. 1-8.
            #3154927925077612244, #Paris-Nice 2023. 1-8.
            #3154928186694035156, 3154928344131990228, #Paris-Nice 2023. 1-8.
            1613964, #Milan San Remo
            169843, #Paris Roubaix
            #1240865, #Amstel Gold Race
            4111919, #Strada Bianche
            16962557, #Liege Bastone Liege
            1402092 #Ronde van Vlaanderen
            ]


In [6]:
# Segment requests
base = 'https://www.strava.com/api/v3/routes/'
for x in route_lst:
    link = base + str(x)
    add_route_to_df(get_request(link))

Done fetching route: Tour de France 2023 - Etape 1 - Bilbao > Bilbao
Done fetching route: Tour de France 2023 - Etape 2 - Donastia San Sebastian > Donastia San Sebastian
Done fetching route: Tour de France 2023 - Etape 3 - Bayonne > Bayonne
Done fetching route: Tour de France 2023 - Etape 4 - Nogaro > Nogaro
Done fetching route: Tour de France 2023 - Etape 5 - Pau > Pau
Done fetching route: Tour de France 2023 - Etape 6 - Tarbes > Tarbes 
Done fetching route: Tour de France 2023 - Etape 7 - Bordeaux > Bordeaux
Done fetching route: Tour de France 2023 - Etape 8 - Limoges > Limoges
Done fetching route: Tour de France 2023 - Etape 9 - Clermont-Ferrand > Clermont-Ferrand
Done fetching route: Tour de France 2023 - Etape 10 - Issoire > Issoire
Done fetching route: Tour de France 2023 - Etape 11 - Moulins > Moulins
Done fetching route: Tour de France 2023 - Etape 12 - Belleville-en-Beaujolais > Belleville-en-Beaujolais
Done fetching route: Tour de France 2023 - Etape 13 - Culoz > Culoz
Done f

In [7]:
route_df.head()

,id,name,segments,distance,elevation_gain,description
0,3017808025791303288,Tour de France 2023 - Etape 1 - Bilbao > Bilbao,"[{'id': 900814, 'resource_state': 2, 'name': '...",80568.344600,1277.838931,81km \n\nD+ : 1275m\n\nDifficulté : 3 étoiles\...
1,3017808470760159864,Tour de France 2023 - Etape 2 - Donastia San S...,"[{'id': 951374, 'resource_state': 2, 'name': '...",51700.763741,938.164881,51km \n\nD+ : 930m \n\nDifficulté : 3 étoiles ...
2,3021095816693259176,Tour de France 2023 - Etape 3 - Bayonne > Bayonne,"[{'id': 900814, 'resource_state': 2, 'name': '...",80568.344600,1277.838931,99km\nD+ : 1000m
3,3028722138306047882,Tour de France 2023 - Etape 4 - Nogaro > Nogaro,"[{'id': 5117861, 'resource_state': 2, 'name': ...",115693.649318,1129.700000,115 km d+=1130m\n\nDifficulté : 2 étoiles\n\n8...
4,3028722407588517896,Tour de France 2023 - Etape 5 - Pau > Pau,"[{'id': 615096, 'resource_state': 2, 'name': '...",130324.034329,2203.672021,130 km d+=2200m\n\nDifficulté : 4 étoiles\n\n5...
